In [17]:
import pandas as pd
import torch
import torchvision
from torchvision.io import read_image
from torchvision.utils import draw_bounding_boxes
from PIL import Image

In [27]:
#Change these paths
elephants_csv = 'sample_elephants_new.csv'
df_elephants = pd.read_csv(elephants_csv)
images = 'sample_images/'
output = 'sample_bbox_images/'
print(df_elephants)

                                  image name     x     y
0   2aada86263161acc6e207ea329759556d308458a   502     2
1   8f30778719e5af075797f994273603c94f170e29    78   176
2   8f30778719e5af075797f994273603c94f170e29   844  2436
3   8f30778719e5af075797f994273603c94f170e29   794  2468
4   8f30778719e5af075797f994273603c94f170e29   687  2432
..                                       ...   ...   ...
67  8b1a6dead4e015f168627d466b5eab09aedbf90d  1785   423
68  8b1a6dead4e015f168627d466b5eab09aedbf90d  1303    18
69  8b1a6dead4e015f168627d466b5eab09aedbf90d  1825   425
70  8b1a6dead4e015f168627d466b5eab09aedbf90d  1779   452
71  8b1a6dead4e015f168627d466b5eab09aedbf90d  1680   505

[72 rows x 3 columns]


In [25]:
#Accumulate rows for new elephants dataframe
data = [] 

#Loop through each image
for image_name in df_elephants['image name'].unique():
    #Get the a dataframe that has all rows with this image
    df_image = df_elephants[df_elephants['image name'] == image_name]

    #Load image
    image_path = f'{images}{image_name}.jpg'
    img = Image.open(image_path) #use PIL to open image so we can save metadata
    metadata = img.getexif()
    #img = read_image(image_path)
    img = torchvision.transforms.functional.pil_to_tensor(img)

    #List of bboxes
    bboxes = []

    #Loop through each elephant in current image to create bboxes
    for index, row in df_image.iterrows():
        #Calculate bbox coordinates assuming x, y is center of elephant
        x = row['x']
        y = row['y']
        width = 25
        height = 25
        x_tl = int(x - width / 2)
        y_tl = int(y - height / 2)
        x_br = int(x + width / 2)
        y_br = int(y + height / 2)

        bbox = [x_tl, y_tl, x_br, y_br]
        bboxes.append(bbox)

        data.append({'image name': image_name, 'xmin': x_tl, 'ymin': y_tl, 'xmax': x_br, 'ymax': y_br})

    #Draw the bboxes
    bboxes = torch.tensor(bboxes, dtype=torch.int)
    img = draw_bounding_boxes(img, bboxes, width=1, colors="red")
    
    #Save the image with bboxes
    img = torchvision.transforms.ToPILImage()(img)
    output_path = f'{output}{image_name}.jpg'
    img.save(output_path, exif=metadata)

In [28]:
#Create new elephants dataframe and output to csv file
df_elephants_new = pd.DataFrame(data)
print(df_elephants_new)

filename = 'sample_elephants_bbox.csv' #Change this filename
df_elephants_new.to_csv(filename, sep=',', index=False)

                                  image name  xmin  ymin  xmax  ymax
0   2aada86263161acc6e207ea329759556d308458a   489   -10   514    14
1   8f30778719e5af075797f994273603c94f170e29    65   163    90   188
2   8f30778719e5af075797f994273603c94f170e29   831  2423   856  2448
3   8f30778719e5af075797f994273603c94f170e29   781  2455   806  2480
4   8f30778719e5af075797f994273603c94f170e29   674  2419   699  2444
..                                       ...   ...   ...   ...   ...
67  8b1a6dead4e015f168627d466b5eab09aedbf90d  1772   410  1797   435
68  8b1a6dead4e015f168627d466b5eab09aedbf90d  1290     5  1315    30
69  8b1a6dead4e015f168627d466b5eab09aedbf90d  1812   412  1837   437
70  8b1a6dead4e015f168627d466b5eab09aedbf90d  1766   439  1791   464
71  8b1a6dead4e015f168627d466b5eab09aedbf90d  1667   492  1692   517

[72 rows x 5 columns]
